In [2]:
import pandas as pd
import datetime
import numpy as np
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle


In [3]:
flights_train = pd.read_csv("../data/flights_cancel_sample.csv",low_memory=False)


In [4]:
flights_train

,Unnamed: 0,fl_date,op_unique_carrier,origin,dest,crs_dep_time,cancelled
0,1521202,2018-07-30,MQ,DCA,XNA,1706,1
1,7329661,2019-06-16,EV,IAH,JAN,1640,1
2,1410788,2018-07-25,MQ,BNA,JFK,757,1
3,2520515,2019-04-12,WN,CMH,MDW,1210,1
4,4328734,2019-07-31,PT,PHL,ALB,2025,1
...,...,...,...,...,...,...,...
538411,13430378,2019-09-22,OO,LIT,IAH,1400,0
538412,4920511,2018-01-31,DL,RDU,DTW,1726,0
538413,9277515,2019-02-12,YX,PIT,PHL,600,1
538414,2521510,2019-04-12,WN,LAS,RNO,605,1


### To predict the cancellation of the flight

In [5]:
def convert_time(df):
    
    df.fl_date = pd.to_datetime(df.fl_date, format='%Y-%m-%d')

    df['month'] = pd.DatetimeIndex(df.fl_date).month

   # df['day'] = pd.DatetimeIndex(df.fl_date).day

    df['day_of_year'] = df.fl_date.dt.dayofyear

    df['day_of_week'] = df.fl_date.dt.dayofweek

    df['crs_dep_time'] = df['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))

    df['dep_hour'] = df['crs_dep_time'].str[:2]

    return df

In [6]:
def bins(df):
    hours = {'00':0,'01':0,'02':0,'03':0,'04':0,'05':0,'06':1,'07':1,'08':1,'09':1,'10':1,'11':2,'12':2,'13':2,'14':2,'15':2,'16':2,'17':3,'18':3,'19':3,'20':4,'21':4,'22':4,'23':4,'24': 4}
    df['dep_part_day'] = df['dep_hour'].map(hours)
    season = {1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3, 8: 3, 9: 4, 10: 4, 11: 4, 12: 1}
    df['season'] = df['month'].map(season)
    
    return df



In [7]:
flights_train.columns

Index(['Unnamed: 0', 'fl_date', 'op_unique_carrier', 'origin', 'dest',
       'crs_dep_time', 'cancelled'],
      dtype='object')

In [8]:
flights_train.head()

,Unnamed: 0,fl_date,op_unique_carrier,origin,dest,crs_dep_time,cancelled
0,1521202,2018-07-30,MQ,DCA,XNA,1706,1
1,7329661,2019-06-16,EV,IAH,JAN,1640,1
2,1410788,2018-07-25,MQ,BNA,JFK,757,1
3,2520515,2019-04-12,WN,CMH,MDW,1210,1
4,4328734,2019-07-31,PT,PHL,ALB,2025,1


In [9]:
df_clf=flights_train[['fl_date', 'op_unique_carrier','origin',  'dest', 'crs_dep_time', 'cancelled']].copy()

In [10]:
df_clf['cancelled'].value_counts()

1    269208
0    269208
Name: cancelled, dtype: int64

In [11]:
missing_values= df_clf.isnull().sum().sort_values(ascending=False)
missing_values

fl_date              0
op_unique_carrier    0
origin               0
dest                 0
crs_dep_time         0
cancelled            0
dtype: int64

In [12]:
df_clf=convert_time(df_clf)
df_clf=bins(df_clf)

df_clf.head()

,fl_date,op_unique_carrier,origin,dest,crs_dep_time,cancelled,month,day_of_year,day_of_week,dep_hour,dep_part_day,season
0,2018-07-30,MQ,DCA,XNA,1706,1,7,211,0,17,3,3
1,2019-06-16,EV,IAH,JAN,1640,1,6,167,6,16,2,3
2,2018-07-25,MQ,BNA,JFK,0757,1,7,206,2,07,1,3
3,2019-04-12,WN,CMH,MDW,1210,1,4,102,4,12,2,2
4,2019-07-31,PT,PHL,ALB,2025,1,7,212,2,20,4,3


In [13]:
df_clf.columns

Index(['fl_date', 'op_unique_carrier', 'origin', 'dest', 'crs_dep_time',
       'cancelled', 'month', 'day_of_year', 'day_of_week', 'dep_hour',
       'dep_part_day', 'season'],
      dtype='object')

In [14]:
X=df_clf[['op_unique_carrier', 'origin', 'dest',
       'month', 'day_of_year', 'day_of_week',
       'dep_hour', 'dep_part_day', 'season']]
y = df_clf.cancelled


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [18]:

from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression()
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier()
from xgboost import XGBClassifier
xgb_clf=XGBClassifier()
from sklearn.metrics import roc_curve, auc, confusion_matrix,accuracy_score,classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [19]:
categorical_vars = ["op_unique_carrier", "origin", "dest"]

# instantiate the one hot encoder
one_hot_encoder = OneHotEncoder(sparse=False, drop = "first")

In [21]:
X=df_clf[['op_unique_carrier', 'origin', 'dest',
       'month', 'day_of_year', 'day_of_week',
       'dep_hour', 'dep_part_day', 'season']]
y = df_clf.cancelled

In [22]:
X_vars_array = one_hot_encoder.fit_transform(X[categorical_vars])
X_vars_df = pd.DataFrame(X_vars_array)
X = pd.concat([X.reset_index(drop=True), X_vars_df.reset_index(drop=True)], axis = 1)
X.drop(categorical_vars, axis = 1, inplace = True)
X

/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,month,day_of_year,day_of_week,dep_hour,dep_part_day,season,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773
0,7,211,0,17,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [23]:
pickle.dump(one_hot_encoder, open('../models/one_hot_encoder_allcancel.pkl', 'wb'))


In [24]:
X.columns = X.columns.astype(str)
X.columns

Index(['month', 'day_of_year', 'day_of_week', 'dep_hour', 'dep_part_day',
       'season', '0', '1', '2', '3',
       ...
       '764', '765', '766', '767', '768', '769', '770', '771', '772', '773'],
      dtype='object', length=780)

In [25]:
X= X.astype('int')

In [26]:
X.dtypes

month           int64
day_of_year     int64
day_of_week     int64
dep_hour        int64
dep_part_day    int64
                ...  
769             int64
770             int64
771             int64
772             int64
773             int64
Length: 780, dtype: object

In [27]:
X.head()

,month,day_of_year,day_of_week,dep_hour,dep_part_day,season,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773
0,7,211,0,17,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=15)

In [29]:
lr_clf=LogisticRegression()
rf_clf=RandomForestClassifier(n_estimators=50, max_depth=10, random_state=25)
xgb_clf=XGBClassifier()


In [30]:
lr_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
xgb_clf.fit(X_train,y_train)

/Users/phanisingaraju/opt/anaconda3/envs/flights_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [31]:
ypred_lr=lr_clf.predict(X_test)
ypred_rf=rf_clf.predict(X_test)
ypred_xgb=xgb_clf.predict(X_test)

In [32]:
print("Accuracy on training set LR: {:.3f}".format(lr_clf.score(X_train,y_train)))
print("Accuracy on testing set LR: {:.3f}".format(accuracy_score(y_test,ypred_lr)))
print("Classification report of LR:\n", classification_report(y_test,ypred_lr))
print("Accuracy on training set RF: {:.3f}".format(rf_clf.score(X_train,y_train)))
print("Accuracy on testing set RF: {:.3f}".format(accuracy_score(y_test,ypred_rf)))
print("Classification report of RF:\n", classification_report(y_test,ypred_rf))
print("Accuracy on training set XGB: {:.3f}".format(xgb_clf.score(X_train,y_train)))
print("Accuracy on testing set XGB: {:.3f}".format(accuracy_score(y_test,ypred_xgb)))
print("Classification report of XGB:\n", classification_report(y_test,ypred_xgb))

Accuracy on training set LR: 0.659
Accuracy on testing set LR: 0.659
Classification report of LR:
               precision    recall  f1-score   support

           0       0.68      0.61      0.64     80822
           1       0.64      0.71      0.68     80703

    accuracy                           0.66    161525
   macro avg       0.66      0.66      0.66    161525
weighted avg       0.66      0.66      0.66    161525

Accuracy on training set RF: 0.656
Accuracy on testing set RF: 0.652
Classification report of RF:
               precision    recall  f1-score   support

           0       0.70      0.54      0.61     80822
           1       0.62      0.76      0.69     80703

    accuracy                           0.65    161525
   macro avg       0.66      0.65      0.65    161525
weighted avg       0.66      0.65      0.65    161525

Accuracy on training set XGB: 0.763
Accuracy on testing set XGB: 0.754
Classification report of XGB:
               precision    recall  f1-score   

In [33]:
pickle.dump(lr_clf, open('../models/lr_app_clf_sampling_allcancel.pkl', 'wb'))
pickle.dump(rf_clf, open('../models/rf_app_clf_sampling.pkl_allcancel', 'wb'))
pickle.dump(xgb_clf, open('../models/xgb_app_clf_sampling_allcancel.pkl', 'wb'))


In [34]:
ypred_xgb_prob=xgb_clf.predict_proba(X_test)

In [35]:
ypred_xgb_prob

array([[0.3704604 , 0.6295396 ],
       [0.12338322, 0.8766168 ],
       [0.8727013 , 0.12729874],
       ...,
       [0.5485767 , 0.45142332],
       [0.32345617, 0.67654383],
       [0.37904775, 0.62095225]], dtype=float32)

In [113]:
prediction = np.argmax(ypred_xgb_prob[0])
proba = round(ypred_xgb_prob[0][prediction], 2)
proba

0.72

In [39]:
type(X_test)

pandas.core.frame.DataFrame